In [1]:
import hashlib

import socket
import os
import subprocess
import pickle
import sys
import threading 
import random
import queue 

In [2]:
def return_hash_value(ip, port):
    name = ip + str(port)
    return int(hashlib.sha256(name.encode('utf-8')).hexdigest(), 16) % max_ring_size

In [3]:
class files_transfer_info():
    def __init__(self, ip, port, hash_val, filename, total_size):
        self.ip_port = (ip, port)
        self.hash_val = hash_val
        self.filename = filename
        self.total_size = total_size
        self.bytes_transferred = 0
    
    def update_record(self, bytes_transferred):
        self.bytes_transferred += bytes_transferred
    
    def return_bytes_transferred(self):
        return self.bytes_transfereed
    
    def check_(self,ip, port, filename, hash_):
        if (self.ip != ip or self.port != port or self.filename != filename 
        or self.hash_val != hash_):
            return False
        else:
            return True
               

In [4]:
class finger_table_entry():
    def __init__(self, hash_calculated):
        self.hash_calculated = hash_calculated
        
    def update_entry(self, ip, port, hash_actual):
        self.ip = ip
        self.port = port
        self.hash_actual = hash_actual
        self.ip_port = (ip, port)
    
    def print_entry(self):
        print (self.hash_calculated, "  " , self.hash_actual)
    
    def print_test(self):
        print (self.hash_calculated)
    
class finger_table():
    def __init__(self, current_node_hash, table_size, max_ring_size):
        self.table = []
        self.table_size = table_size
        self.max_ring_size = max_ring_size
        for i in range(table_size):
            entry = finger_table_entry((current_node_hash + 2**i) % max_ring_size)
            self.table.append(entry)
    
    def return_calculated_value(self, index):
        return self.table[index].hash_calculated
    
    def return_ip_port(self, index):
        return self.table[index].ip_port
    
    def update_entry(self, index, ip, port, hash_actual):
        self.table[index].update_entry(ip, port, hash_actual)
        
    def print_table(self):
        for i in range(self.table_size):
            self.table[i].print_entry()
            
    def print_testing(self):
         for i in range(self.table_size):
            self.table[i].print_test()       

In [5]:
class server():
    def __init__(self, ip, port):
        self.host = ip
        self.port = port
        
        self.create_socket()

    def create_socket(self):
        try:
            self.skt = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            self.skt.setsockopt( socket.SOL_SOCKET, socket.SO_REUSEADDR, 1 )
        except socket.error as msg:
            print ("Socket creation error: " + str(msg))
            return

        try:
            print ("Binding the self.port " + str(self.port))
            self.skt.bind((self.host, self.port))   
            print ("Binding done")
        except socket.error as msg:
            print ("Socket binding error: " + str(msg) + "\n" + "Retrying...")
            return
        
        
    def socket_accept(self):
        print ("listen")
        self.skt.listen(2)
        self.conn, self.address = self.skt.accept()  #first output gives object of connection, second gives (ip address + self.port) of client
        print ("Connection has been established!\n" + "IP: " + self.address[0] + "   port: " + str(self.address[1]))
        
        self.connected_to_IP = self.address[0]
        self.connected_to_port = self.address[1]
        hash_value_str = self.connected_to_IP + str(self.connected_to_port)
        self.hash_value = return_hash_value(self.connected_to_IP, str(self.connected_to_port))
    
    def change_IP_port(self, IP, port):
        self.host = IP
        self.port = port
        
    def start(self):
        self.socket_accept()
        return (self.conn, self.address)
#         self.info = (self.conn, self.host, self.port)
#         return self.info
    
    
class client():
    def __init__(self, ip, port):
        self.host = ip
        self.port = port
        
    def create_connection(self):    
        try:
            self.skt = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            self.skt.setsockopt( socket.SOL_SOCKET, socket.SO_REUSEADDR, 1 )
            print ("trying to connect")
            self.skt.connect((self.host,self.port))
            print ("connected")
        except socket.error as msg:
            print ("Socket connection error: " + str(msg))
            
        
    def start(self):
        self.create_connection()
#         self.info = (self.skt, self.host, self.port)
#         return self.info
        return self.skt

In [6]:
class node():
    def __init__(self):
        self.ip = '192.168.18.94'
        self.port = port_test
        self.hash_value = return_hash_value(self.ip, self.port)
        self.max_ring_size = max_ring_size
        self.table_size = m
        self.predecessor = (self.ip, self.port, self.hash_value)
        
        self.files_receiving_record = []
        self.files_sending_record = []                        
        self.finger_table = finger_table(self.hash_value, self.table_size, self.max_ring_size)
        
        self.file_list = []
        self.update_file_list()
        self.start(first_node)
        
        print ("Own Hash: ", self.hash_value)
        
    def start(self, firstNode=False):
        self.num_of_servers = 1
        server1 = server(self.ip, self.port)
        for i in range(self.num_of_servers):
            f =  threading.Thread(target = self.start_single_server, args = ([server1]) )
            f.start()
        
        if firstNode == False:
            ip = self.ip
            port = connect_to
            self.send_join_request(ip, port)
        else:
            for i in range(self.table_size):
                self.finger_table.update_entry(i, self.ip, self.port, self.hash_value)
            
            
    def start_single_server(self, server1):
        conn, address = server1.start()        
        mode = str(conn.recv(1024), "utf-8")
        
        if mode == 'join':
            self.recv_join_request(conn, address) #1
        elif mode == 'find successor':
            print ("Find sucessor request recieved")
            conn.send(str.encode('ack')) #2
            hash_target = pickle.loads(conn.recv(1024)) #3
            ip_port =self.find_successor(hash_target)
            conn.send(pickle.dumps(ip_port)) #4  
        #asking about current nodes predecessor
        elif mode == 'pred_info':
            print ("Predeccesor info request recieved")
            conn.send(pickle.dumps(self.predecessor))
        #telling that im your predecessor
        elif mode == 'update_pred':
            print ("Update predecessor request recieved")
            conn.send(str.encode('ack')) #2
            pred = pickle.loads(conn.recv(1024)) #3
            self.predecessor = pred
            self.transfer_files_to_new_predecessor(conn)
        #telling that im your sucessor
        elif mode == 'update_succ':
            print ("Update sucessor request recieved")
            conn.send(str.encode('ack')) #2
            succ_ip, succ_port, succ_hash = pickle.loads(conn.recv(1024)) #3
            self.finger_table.update_entry(0, succ_ip, succ_port, succ_hash)
        elif mode == 'file_request':
            conn.send(str.encode('ack'))
            ip1, port1 = pickle.loads(conn.recv(1024))
            conn.send(str.encode('ack'))
            self.send_file(conn, ip1, port1)
        elif mode == 'file_store':  
            conn.send(str.encode('ack'))
            self.save_file(conn) 
        elif mode == 'leave':
            conn.send(str.encode('ack')) #2
            num = str(conn.recv(1024), "utf-8") #3
            num = int(num)
            for i in range(num):
                self.save_file(conn)
                
            conn.send(str.encode('ack')) #2
            pred = pickle.loads(conn.recv(1024)) #3
            self.predecessor = pred
            self.transfer_files_to_new_predecessor(conn)
            
        else:
            print ("recieved random data")
            
        conn.close()
        
        print ("Predecessor info: ", self.predecessor)
        self.update_finger_table()
        self.finger_table.print_table()
        self.start_single_server(server1)
        
        
    def get_predecessor_info(self, ip_succ, port_succ):
        conn = self.start_client(ip_succ, port_succ)
        conn.send(str.encode('pred_info'))
        pred_ip_port_hash = pickle.loads(conn.recv(1024))
        conn.close()
        return pred_ip_port_hash
    
    def send_join_request(self, ip1, port1):
        ip = ip1
        port = port1 
        
        print ("Sending join request")
        
        conn = self.start_client(ip, port)
        conn.send(str.encode('join'))
        
        successor_IP, successor_port = pickle.loads(conn.recv(1024)) #info should contain (hashvalue, ip, port)
        successor_hash = return_hash_value(successor_IP, successor_port)
        print ("\nSetting new Sucessor", successor_hash)
        conn.close()
        
        #making successors predecessor our predecessor
        self.predecessor = self.get_predecessor_info(successor_IP, successor_port)
        print ("\nSetting new Predecessor")
        print (self.hash_value , " pred = ", self.predecessor[2])
        
        #tell sucessor we are his new predecessor
        conn = self.start_client(successor_IP, successor_port)
        conn.send(str.encode('update_pred')) #1
        rec = str(conn.recv(1024), "utf-8")  #2
        info = (self.ip, self.port, self.hash_value)
        conn.send(pickle.dumps(info)) #3
        
        num_of_files = str(conn.recv(1024), "utf-8")#4
        num_of_files = int(num_of_files)
        
        for i in range(num_of_files):
            self.save_file(conn)
        
        conn.close()
        
        #telling our pred to make us their sucessor
        pred_ip, pred_port, pred_hash = self.predecessor
        conn = self.start_client(pred_ip, pred_port)
        conn.send(str.encode('update_succ')) #1
        rec = str(conn.recv(1024), "utf-8")  #2
        info = (self.ip, self.port, self.hash_value)
        conn.send(pickle.dumps(info)) #3
        conn.close()
            
        for i in range(self.table_size):
            self.finger_table.update_entry(i, successor_IP, successor_port, successor_hash)
            
#         self.update_finger_table()

        
    def start_client(self, ip, port):
        c1 = client(ip, port)
        conn_client = c1.start()
        return conn_client
    
    def recv_join_request(self, conn, address):
        print ("Recieved join request")
        hash_target = return_hash_value(address[0], address[1])
        ip_port = self.find_successor(hash_target)
        
        conn.send(pickle.dumps(ip_port))
        
    def update_finger_table(self):
#         print ("\n\nUpdate finger table called")
        for i in range(self.table_size):
            hash_target = self.finger_table.return_calculated_value(i)
#             print ("Hash value to find successor of: ", hash_target)
            ip, port = self.find_successor(hash_target)
            hash_actual = return_hash_value(ip, port)
#             print ("Actual hash value: ", hash_actual)
            self.finger_table.update_entry(i, ip, port, hash_actual)

        
    def distance(self, node1, node2):
        if node1 == node2:
            return 0
        elif node1 < node2:
            return node2 - node1
        else:
            return self.max_ring_size - node1 + node2  
    
    def between(self, node1, node2, target_hash):
        if self.distance(node1, target_hash) <= self.distance(node2, target_hash):                                                               
            return True
        else:
            return False

    def find_successor(self, target_hash):
        if self.hash_value == target_hash:
            return (self.ip, self.port)
        elif (self.between(self.hash_value, self.finger_table.table[0].hash_actual, target_hash)==True):
            return self.finger_table.table[0].ip_port
        else:
            for i in range(self.table_size-1):
                if (self.between(self.finger_table.table[i].hash_actual, 
                            self.finger_table.table[i+1].hash_actual, target_hash) == True):
                    
                    ip, port = self.finger_table.return_ip_port(i)
                    conn = self.start_client(ip, port)
                    conn.send(str.encode('find successor')) #1
                    response = str(conn.recv(1024), "utf-8") #2
                    conn.send(pickle.dumps(target_hash)) #3
                    return pickle.loads(conn.recv(1024)) #4
    
            ip, port = self.finger_table.return_ip_port(-1)
            conn = self.start_client(ip, port)
            conn.send(str.encode('find successor')) #1
            response = str(conn.recv(1024), "utf-8") #2
            conn.send(pickle.dumps(target_hash)) #3
            return pickle.loads(conn.recv(1024)) #4  

    def leave_network(self):  
        succ_ip, succ_port = self.finger_table.return_ip_port(0)
        skt = self.start_client(succ_ip, succ_port)
        skt.send(str.encode('leave')) #1
        ack = str(skt.recv(1024), "utf-8") #2
        self.update_file_list()
        num = str(self.num_of_files)
        skt.send(str.encode(num)) #3
        
        #send all files
        for i in range(self.num_of_files):
            self.send_file_to_given_node(succ_ip, succ_port, self.file_list[i][0], skt)
        
        #tell successor to update pred
        ack = str(skt.recv(1024), "utf-8")
        skt.send(pickle.dumps(self.predecessor))
        skt.close()
        
        #tell pred to update succ
        try:
            skt = self.start_client(self.predecessor[0], self.predecessor[1])
            skt.send(str.encode('update_succ')) #1
            ack = str(conn.recv(1024), "utf-8")
            succ_hash = return_hash_value(succ_ip, succ_port)
            info = (succ_ip, succ_port, succ_hash)
            skt.send(pickle.dumps(info))
        except:
            pass
        
        
    def save_file(self, skt):
        filename, file_size = pickle.loads(skt.recv(1024)) #1
        skt.send(str.encode("OK")) #2
        total_bytes_recieved = 0
        
        file = open(' '+filename, 'wb')
        data = skt.recv(1024) #3
        total_bytes_recieved += len(data)
        file.write(data)
        
        interval = file_size//10
        counter = 0
        while (total_bytes_recieved < file_size):
            data = skt.recv(1024) #4
            total_bytes_recieved += len(data)
            file.write(data)
            if counter%interval == 0:
                print ((total_bytes_recieved/file_size)*100, "% Done")
            counter += 1
        print ("Saving file complete!")
        
        
    def store_file(self):
        filename = input("Enter the name of the file you want to store:  ")
        skt = self.start_client(succ_ip, succ_port)
        if filename != 'q':
            if os.path.isfile(filename):
                file_hash = return_hash_value(filename, " ")
                succ_ip, succ_port = self.find_successor(file_hash) 

                skt.send(str.encode('file_store')) 
                ack = skt.recv(1024)

                file_size = os.path.getsize(filename)
                skt.send(pickle.dumps((filename, file_size))) #1
                response = str(skt.recv(1024), "utf-8")   #2
                with open(filename, 'rb') as file:
                    total_bytes_sent = 0
                    bytes_to_send = file.read(1024)
                    total_bytes_sent += len(bytes_to_send)
                    skt.send(bytes_to_send) #3
                    interval = file_size//10
                    counter = 0
                    while total_bytes_sent < file_size:
                        bytes_to_send = file.read(1024)
                        total_bytes_sent += len(bytes_to_send)
                        skt.send(bytes_to_send)  #4
                        if counter%interval == 0:
                            print ((total_bytes_sent/file_size)*100, "% uploaded")
                        counter += 1
                    print ("File uploaded!")
                os.remove(filename)
            else:
                skt.send(str.encode("Does not exist"))
            skt.close()
            
            
    
    
    def check_if_file_in_record(self, ip, port, filename, hash_ , record):
        for i in range(len(record)):
            if record[i].check_(ip, port, filename, hash_) == True:
                return i
        return -1
                
             
    def send_file(self, skt, ip, port):
        filename, bytes_already_rec_by_client = pickle.loads(skt.recv(1024)) #1
        file_size = os.path.getsize(filename)
        hash_v = return_hash_value(filename, " ")
        file_index = self.check_if_file_in_record(ip, port, filename, hash_v, self.files_sending_record)
        if file_index >= 0:
            bytes_already_sent_in_our_record = self.files_sending_record[file_index].return_files_transferred()
        else:
            bytes_already_sent_in_our_record = 0 
            file_index = len(self.files_sending_record)
            file_info = files_transfer_info(ip, port, hash_v, filename, file_size)
            self.files_sending_record.append(file_info)

        bytes_already_sent = bytes_already_sent_in_our_record
        if (bytes_already_sent_in_our_record != bytes_already_rec_by_client):
            bytes_already_sent = bytes_already_rec_by_client

        if os.path.isfile(filename):
            skt.send(str.encode("Exists" + str(file_size))) #2
            client_response = str(skt.recv(1024), "utf-8") #3
            with open(filename, 'rb') as file:
                total_bytes_sent = 0
                if bytes_already_sent > 0:
                    skt.send(str.encode('Pre' + str(bytes_already_sent)))    #4        
                    client_response = str(skt.recv(1024), "utf-8") #5
                    if client_response == 'y' or 'Y':
                        already_sent = file.read(bytes_already_sent)  #if we need to resume
                        total_bytes_sent = bytes_already_sent     
                else:
                    skt.send(str.encode("Downloading file for the first time")) #4
                    client_response = str(skt.recv(1024), "utf-8") #5

                bytes_to_send = file.read(1024)
                total_bytes_sent += len(bytes_to_send)
                self.files_sending_record[file_index].update_record(total_bytes_sent)
                
                skt.send(bytes_to_send) #6
                while bytes_to_send != "":
                    bytes_to_send = file.read(1024)
                    total_bytes_sent += len(bytes_to_send)
                    self.files_sending_record[file_index].update_record(total_bytes_sent)
                    skt.send(bytes_to_send) 
                
                del self.files_sending_record[file_index]
        else:
            skt.send(str.encode("Does not exist"))
    
    def retrieve_file(self):
        filename = input("Enter the name of the file you want to retrieve:  ")
        if filename != 'q':
            file_hash = return_hash_value(filename, " ")
            succ_ip, succ_port = self.find_successor(file_hash)
            
            skt = self.start_client(succ_ip, succ_port)
            skt.send(str.encode('file_request'))
            ack = skt.recv(1024)
            skt.send(pickle.dumps((self.ip, self.port)))
            ack = skt.recv(1024)
            self.save_file_helper(skt, succ_ip, succ_port, filename)
            
    def save_file_helper(self, skt, ip, port, filename):
        if filename != 'q':
            hash_v = return_hash_value(filename, " ")
            file_index = self.check_if_file_in_record(ip, port, filename, hash_v, self.files_receiving_record)
            if file_index >= 0:
                already_receieved_bytes = self.files_sending_record[file_index].return_files_transferred()
            else:
                already_receieved_bytes = 0 
                file_index = len(self.files_receiving_record)
                file_info = files_transfer_info(ip, port, hash_v, filename, 0) ####000??? change
                self.files_receiving_record.append(file_info)
                
            skt.send(pickle.dumps((filename, already_receieved_bytes)))

            response = str(skt.recv(1024), "utf-8") #2
            if response[0:6] == 'Exists':
                file_size = int(response[6:])
                msg = input("File exists. \nSize: " + str(file_size) + 
                            " Bytes\nDo you want to download? (y/n)") 
                if msg == 'y' or msg == 'Y':
                    skt.send(str.encode('OK')) #3
                    response = str(skt.recv(1024), "utf-8") #4
                    if response[:3] == 'Pre':
                        inp = input("You have already downloaded till " + str(response[3:]) 
                                       + " bytes\n Do you want to continue? (y/n)")
                        if inp == 'y' or inp == 'Y':
                            skt.send(str.encode(inp)) #5
                            total_bytes_recieved = int(response[3:])
                        else:
                            skt.send(str.encode("n")) #5
                            total_bytes_recieved = 0
                    else:
                        total_bytes_recieved = 0
                        skt.send(str.encode("ok")) #5


                    file = open(' '+ filename, 'wb')
                    data = skt.recv(1024) #6
                    total_bytes_recieved += len(data)
                    self.files_receiving_record[file_index].update_record(total_bytes_recieved)
                    file.write(data)
                    counter = 0
                    interval = file_size//10
                    while (total_bytes_recieved < file_size):
                        data = skt.recv(1024) #6
                        total_bytes_recieved += len(data)
                        self.files_receiving_record[file_index].update_record(total_bytes_recieved)
                        file.write(data)
                        if counter%interval==0:
                            print ((total_bytes_recieved/file_size)*100, "% Done")
                        counter += 1
                    del self.files_receiving_record[file_index]
                    print ("Download complete!")
            else:
                print("File does not exist")

            skt.close()

            
    def update_file_list(self):
        self.file_list = os.listdir()
        self.num_of_files = len(self.file_list)
        index_to_delete = []
        
        for i in range(self.num_of_files):
            if '.ipynb' in self.file_list[i] or '.DS_Store' in self.file_list[i]:
                index_to_delete.append(i)
        
        for i in range(len(index_to_delete)):
            del self.file_list[index_to_delete[len(index_to_delete)-1-i]]
        
        self.num_of_files = len(self.file_list)
        for i in range(self.num_of_files):
            file_hash = return_hash_value(self.file_list[i], " ")
            self.file_list[i] = (self.file_list[i], file_hash)
            
    def print_file_list(self):
        self.update_file_list()
        for i in range(self.num_of_files):
            print (self.file_list[i])
            
    def transfer_files_to_new_predecessor(self, skt):
        self.update_file_list()
        files_to_transfer = []
        for i in range(self.num_of_files):
#             print ("\n\n\nChecking between")
#             print ("Pred hash: ", self.predecessor[2])
#             print ("own hash: ", self.hash_value)
#             print ("file hash: ", self.file_list[i][1])
            if self.between(self.predecessor[2], self.hash_value, self.file_list[i][1]):
                files_to_transfer.append(self.file_list[i][0])
        
        num = str(len(files_to_transfer))
        skt.send(str.encode(num))
        
        for i in range(len(files_to_transfer)):
            self.send_file_to_given_node(self.predecessor[0], self.predecessor[1], files_to_transfer[i], skt)
               
                
    def send_file_to_given_node(self, ip, port, filename, skt):
                file_size = os.path.getsize(filename)
                skt.send(pickle.dumps((filename, file_size))) #1
                response = str(skt.recv(1024), "utf-8")   #2
                with open(filename, 'rb') as file:
                    total_bytes_sent = 0
                    bytes_to_send = file.read(1024)
                    total_bytes_sent += len(bytes_to_send)
                    skt.send(bytes_to_send) #3
                    counter = 0
                    interval = file_size//10
                    while total_bytes_sent < file_size:
                        bytes_to_send = file.read(1024)
                        total_bytes_sent += len(bytes_to_send)
                        skt.send(bytes_to_send)  #4
                        if counter%interval == 0:
                            print ((total_bytes_sent/file_size)*100, "% uploaded")
                        counter += 1
                    print ("File uploaded!")
                os.remove(filename) 
                
    
        

In [8]:
m = 6
max_ring_size = 2**m
port_test = 40285
first_node = True
connect_to = 50342

n = node()